In [1]:
import numpy as np
np.matrix # <-- refrence, I suppose


'\nGoal - make a Matrix class\ngive it some magic methods - list later on\nmake a "Transpose" function\n\nList of magic methods\n__new__(cls, other) \n__del__(self) \n__abs__(self)\n__add__(self, other) \tTo get called on add operation using + operator\n__sub__(self, other) \tTo get called on subtraction operation using - operator.\n__mul__(self, other) \tTo get called on multiplication operation using * operator.\n__floordiv__(self, other) \tTo get called on floor division operation using // operator.\n__truediv__(self, other) \tTo get called on division operation using / operator.\n__mod__(self, other) \tTo get called on modulo operation using % operator.\n__pow__(self, other[, modulo]) \tTo get called on calculating the power using ** operator.\n__lt__(self, other) \tTo get called on comparison using < operator.\n__le__(self, other) \tTo get called on comparison using <= operator.\n__eq__(self, other) \tTo get called on comparison using == operator.\n__ne__(self, other) \tTo get cal

In [2]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix

    def __len__(self):
        return len(self.matrix)

    def get_dimensions(self):
        rows = len(self.matrix)
        columns = len(self.matrix[0]) if rows > 0 else 0
        return rows, columns

    @staticmethod
    def is_same_dimensions(matrix1, other_matrix):
        return len(matrix1.matrix) == len(other_matrix.matrix) and len(matrix1.matrix[0]) == len(other_matrix.matrix[0])

    def __add__(self, other_matrix):
        new_matrix = []
        if isinstance(other_matrix, int) or isinstance(other_matrix, float):
            for row in self.matrix:
                new_matrix.append([])
                for col in row:
                    new_matrix[-1].append(col + other_matrix)
        else:
            for row in zip(self.matrix, other_matrix.matrix):
                new_matrix.append([])
                for i in range(len(row[0])):
                    new_matrix[-1].append(row[0][i] + row[1][i])
        return Matrix(new_matrix)

    def T(self):
        rows = len(self)
        columns = len(self.matrix[0]) if rows > 0 else 0

        transposed = []
        for j in range(columns):
            row = []
            for i in range(rows):
                row.append(self.matrix[i][j])
            transposed.append(row)

        return Matrix(transposed)

    def __eq__(self, other_matrix):
        return self.is_same_dimensions(self, other_matrix) and all(
            self.matrix[i][j] == other_matrix.matrix[i][j]
            for i in range(len(self))
            for j in range(len(self.matrix[i]))
        )

    def __repr__(self):
        return repr(self.matrix)

    def __sub__(self, other):
        new_matrix = []
        if isinstance(other, int) or isinstance(other, float):
            for row in self.matrix:
                new_matrix.append([])
                for col in row:
                    new_matrix[-1].append(col-other)
        else:
            if not self.is_same_dimensions(self, other):
                raise ValueError("Матрицы должны иметь одинаковые размерности")
            for row in zip(self.matrix, other.matrix):
                new_matrix.append([])
                for i in range(len(row[0])):
                    new_matrix[-1].append(row[0][i]-row[1][i])
        return Matrix(new_matrix)


    def __mul__(self, other_matrix):
        
        result = []
        if isinstance(other_matrix, int) or isinstance(other_matrix, float):
            for i in self.matrix:
                result.append([])
                for j in i:
                    result[-1].append(j*other_matrix)

        else:
            rows, columns = self.get_dimensions()
            other_rows, other_columns = other_matrix.get_dimensions()
            if columns != other_rows:
                raise ValueError("Матрицы должны иметь одинаковые размерности")

         
            if not self.is_same_dimensions(self, other_matrix):
                raise ValueError("Матрицы должны иметь одинаковые размерности")
            for row in zip(self.matrix, other_matrix.matrix):
                result.append([])
                for i in range(len(row[0])):
                    result[-1].append(row[0][i]*row[1][i])

        return Matrix(result)

    def __truediv__(self, other_matrix):
        result = []
        if isinstance(other_matrix, int) or isinstance(other_matrix, float):
            for row in self.matrix:
                result.append([])
                for col in row:
                    result[-1].append(col/other_matrix)
        else:
            if not self.is_same_dimensions(self, other_matrix):
                raise ValueError("Матрицы должны иметь одинаковые размерности")

            result = [
                [self.matrix[i][j] / other_matrix.matrix[i][j] for j in range(len(self.matrix[i]))]
                for i in range(len(self.matrix))
            ]

        return Matrix(result)

    def __matmul__(self, other_matrix):
        if len(self.matrix[0]) != len(other_matrix.matrix):
            raise ValueError("Матрицы должны иметь согласованные размерности")

        result = [[sum(self.matrix[i][k] * other_matrix.matrix[k][j] for k in range(len(self.matrix[0])))
                   for j in range(len(other_matrix.matrix[0]))]
                  for i in range(len(self.matrix))]

        return Matrix(result)

In [4]:
%pip install -q ipytest pytest

import ipytest
import pytest
ipytest.autoconfig()

Note: you may need to restart the kernel to use updated packages.


In [5]:
%%ipytest
import numpy as np

matrix1 = Matrix([[1,2,3],[1,2,3],[1,2,3]])
matrix2 = Matrix([[4,5,6],[4,5,6],[4,5,6]])

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_add_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 + matrix2).matrix) == (np.array(matrix1.matrix) + np.array(matrix2.matrix))).all()
    ), 'add section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13.2]])
def test_matrix_add_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 + number).matrix) == np.array(matrix1.matrix) + number).all()
    ), 'add section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_sub_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 - matrix2).matrix) == np.array(matrix1.matrix) - np.array(matrix2.matrix)).all()
    ), 'sub section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 12.2]])    
def test_matrix_sub_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 - number).matrix) == np.array(matrix1.matrix) - number).all()
    ), 'sub section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1", [matrix1])    
def test_matrix_transpose(matrix1: Matrix):
    assert (
        (np.array(matrix1.T().matrix) == np.array(matrix1.matrix).T).all()
    ), 'transpose section is not working right'
    
# -----------------------------------------

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 * matrix2).matrix) == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
    ), 'mul section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13]])
def test_matrix_mul_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 * number).matrix) == np.array(matrix1.matrix) * number).all()
    ), 'mul section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])    
def test_matrix_div_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 / matrix2).matrix) == np.array(matrix1.matrix) / np.array(matrix2.matrix)).all()
    ), 'div section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 2]])    
def test_matrix_div_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 / number).matrix) == np.array(matrix1.matrix) / number).all()
    ), 'div section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2.T()]])    
def test_matrix_matmul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert(
        (np.array((matrix1 @ matrix2).matrix) == np.array(matrix1.matrix) @ np.array(matrix2.matrix)).all()
    ), 'matmul section is not working right, and don\'t forget about T() if needed'

print((matrix1*matrix2).matrix)
print(np.array(matrix1.matrix) * np.array(matrix2.matrix))


[[4, 10, 18], [4, 10, 18], [4, 10, 18]]
[[ 4 10 18]
 [ 4 10 18]
 [ 4 10 18]]
..........                                                                                   [100%]
10 passed in 0.03s
